<a href="https://www.kaggle.com/code/niramay/obama-quotes-classification?scriptVersionId=112749264" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Activation
from keras.layers import Input, TextVectorization
from keras.callbacks import *
from keras.metrics import *
from keras.backend import clear_session
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

import spacy

In [2]:
data = pd.read_csv('/kaggle/input/obama-quotestruths-lies/politifact-obama.csv')
data.head()

,Score,Date,Quote
0,False,"July 22, 2021","""The cost of an automobile, it's kind of back ..."
1,False,"June 23, 2021","""The Second Amendment, from the day it was pas..."
2,False,"May 3, 2021",For vaccine rates among Americans 65 and older...
3,False,"March 25, 2021",“We’re sending back the vast majority of the f...
4,False,"February 16, 2021","""If we kept (the minimum wage) indexed to infl..."


In [3]:
data['Score'].value_counts()

Half True        211
Mostly True      211
True             148
False            107
Mostly False     101
Pants on Fire     15
Score              1
Name: Score, dtype: int64

In [4]:
data.drop(['Date'], axis=1, inplace=True)
data.head()

,Score,Quote
0,False,"""The cost of an automobile, it's kind of back ..."
1,False,"""The Second Amendment, from the day it was pas..."
2,False,For vaccine rates among Americans 65 and older...
3,False,“We’re sending back the vast majority of the f...
4,False,"""If we kept (the minimum wage) indexed to infl..."


In [5]:
data['check'] = data['Score'].replace({"Mostly True": 1, "True":1, "Half True":0, "Mostly False":0,"False":0,"Pants on Fire":0,"Score":0})
data['check'].value_counts()

0    435
1    359
Name: check, dtype: int64

In [6]:
data.drop('Score', axis=1, inplace = True)
data.head()

,Quote,check
0,"""The cost of an automobile, it's kind of back ...",0
1,"""The Second Amendment, from the day it was pas...",0
2,For vaccine rates among Americans 65 and older...,0
3,“We’re sending back the vast majority of the f...,0
4,"""If we kept (the minimum wage) indexed to infl...",0


In [7]:
X = data['Quote']
y = data['check']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((555,), (239,), (555,), (239,))

In [8]:
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(X_train))
words_to_index = tokenizer.word_index
#converting text into integer sequences
X_train_seq  = tokenizer.texts_to_sequences(X_train) 
X_test_seq = tokenizer.texts_to_sequences(X_test)

#padding to prepare sequences of same length
X_train_seq  = pad_sequences(X_train_seq, maxlen=100)
X_test_seq = pad_sequences(X_test_seq, maxlen=100)

In [9]:
size_of_vocabulary=len(tokenizer.word_index) + 1
print(size_of_vocabulary)

2285


In [10]:
METRICS = [
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR'), # precision-recall curve  
]

2022-12-02 11:44:55.977150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 11:44:55.978347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 11:44:55.979013: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 11:44:55.979821: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [11]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip


--2022-12-02 11:45:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-12-02 11:45:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-12-02 11:45:05--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [12]:
size_of_vocabulary=len(tokenizer.word_index) + 1
size_of_vocabulary

2285

In [13]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [14]:
word_to_vec_map = read_glove_vector('/kaggle/working/glove.6B.50d.txt')


In [15]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=size_of_vocabulary, weights = [emb_matrix], trainable=False)

In [16]:
clear_session()

model=Sequential()

#embedding layer
model.add(embedding_layer) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=METRICS) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)  
mc=ModelCheckpoint('best_model', monitor='val_loss', mode='min', save_best_only=True, save_weights_only=False, verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2285, 50)          114200    
_________________________________________________________________
lstm (LSTM)                  (None, 2285, 128)         91648     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 214,169
Trainable params: 99,969
Non-trainable params: 114,200
_________________________________________________________________
None


In [17]:
history = model.fit(np.array(X_train_seq),np.array(y_train),batch_size=128,epochs=10,
                    validation_data=(np.array(X_test_seq),np.array(y_test)),verbose=1,callbacks=[es,mc])

2022-12-02 11:48:11.398045: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-12-02 11:48:14.588251: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


5/5 [==============================] - 6s 288ms/step - loss: 0.6909 - precision: 0.4444 - recall: 0.0813 - auc: 0.4888 - prc: 0.4425 - val_loss: 0.6876 - val_precision: 1.0000 - val_recall: 0.0088 - val_auc: 0.5965 - val_prc: 0.5658

Epoch 00001: val_loss improved from inf to 0.68759, saving model to best_model


2022-12-02 11:48:18.934343: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 2/10
5/5 [==============================] - 0s 19ms/step - loss: 0.6807 - precision: 0.8571 - recall: 0.0732 - auc: 0.5735 - prc: 0.5445 - val_loss: 0.6826 - val_precision: 0.8750 - val_recall: 0.0619 - val_auc: 0.6210 - val_prc: 0.6040

Epoch 00002: val_loss improved from 0.68759 to 0.68259, saving model to best_model
Epoch 3/10
5/5 [==============================] - 0s 19ms/step - loss: 0.6758 - precision: 0.9286 - recall: 0.0528 - auc: 0.6052 - prc: 0.5881 - val_loss: 0.6802 - val_precision: 0.7391 - val_recall: 0.1504 - val_auc: 0.6136 - val_prc: 0.6034

Epoch 00003: val_loss improved from 0.68259 to 0.68024, saving model to best_model
Epoch 4/10
5/5 [==============================] - 0s 18ms/step - loss: 0.6659 - precision: 0.7872 - recall: 0.1504 - auc: 0.6575 - prc: 0.6139 - val_loss: 0.6756 - val_precision: 0.5844 - val_recall: 0.3982 - val_auc: 0.6168 - val_prc: 0.6045

Epoch 00004: val_loss improved from 0.68024 to 0.67562, saving model to best_model
Epoch 5/10
5/5 [===

In [18]:
from keras.models import load_model
model = load_model('best_model')

In [19]:
model_results = model.evaluate(X_test_seq, y_test, batch_size=128, verbose=0)

for name, value in zip(model.metrics_names, model_results):
  print(name, ': ', value)

loss :  0.6749681234359741
precision :  0.5853658318519592
recall :  0.21238937973976135
auc :  0.6108301281929016
prc :  0.5866303443908691
